In [1]:
%env WANDB_PROJECT=aspect_explain_vulnerability_type

env: WANDB_PROJECT=aspect_explain_vulnerability_type


In [2]:
# import datasets

# ds = datasets.load_dataset("code_x_glue_ct_code_to_text", 'python')

from project_dataset import load_dataset


In [3]:
from dataclasses import dataclass

@dataclass
class ArgsVulType:
    model_name = "Salesforce/codet5p-220m"
    num_proc = 8
    batch_size = 5
    max_src_length = 1200
    max_des_length = 53
    data_cols = ["CVE ID", "explain", "func_before"]
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "vulnerability_type"
    
args = ArgsVulType()

In [4]:
ds = load_dataset(args.task)

In [5]:
import os

os.makedirs(args.save_dir, exist_ok=True)

In [6]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [7]:
import numpy as np

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()
    # lables = np.where(lables != codet5p_tokenizer.pad_token_id, lables, -100)

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = ds.map(
  preprocess_function,
  remove_columns=args.data_cols,
  batched=True,
  num_proc=args.num_proc,
  batch_size=args.batch_size)

tokenized_ds

Map (num_proc=8):   0%|          | 0/3870 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/431 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1076 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3870
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 431
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1076
    })
})

In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name).to(device)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=model,
  return_tensors="pt")

In [10]:
import evaluate
import numpy as np

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = codet5p_tokenizer(arg)
  return codet5p_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, codet5p_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = codet5p_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = codet5p_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  text_preds = ["\n".join(p for p in text_preds)]
  text_labels = ["\n".join(l for l in text_labels)]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [49]:
# for testing, show first 5 rows

from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()
  break

metrics_func([preds, labels])

{'rouge1': 0.8823529411764706,
 'rouge2': 0.8118811881188118,
 'rougeL': 0.8725490196078433,
 'rougeLsum': 0.8867924528301887}

In [12]:
preds.shape, labels.shape

(torch.Size([1, 53]), torch.Size([1, 53]))

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        report_to='wandb',
        output_dir=args.save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',
        do_eval=True,

        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_acc_steps,

        learning_rate=args.lr,
        weight_decay=args.weight_decay,
        warmup_steps=args.lr_warmup_steps,

        logging_dir=args.save_dir,
        logging_first_step=True,
        logging_steps=args.log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=args.num_proc,

        local_rank=args.local_rank,
        deepspeed=args.deepspeed,
        fp16=args.fp16,
    )

In [15]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    tokenizer = codet5p_tokenizer,
)

trainer.train()

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongchirua. Use `wandb login --relogin` to force relogin


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,8.733200
10,8.768700
20,7.451800
30,6.207700
40,5.749500
50,5.360200
60,4.977300
70,4.692600
80,4.308600
90,3.912900


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-package

TrainOutput(global_step=9600, training_loss=0.11759449704504125, metrics={'train_runtime': 26617.1244, 'train_samples_per_second': 14.54, 'train_steps_per_second': 0.361, 'total_flos': 5.48062101504e+17, 'train_loss': 0.11759449704504125, 'epoch': 99.22})

In [16]:
trainer.save_model(f'{args.save_dir}/{args.task}')

## measure on test set

In [50]:
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=args.batch_size,
  num_workers=args.num_proc)

rouge_list = []

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()

  rouge_result = metrics_func([preds, labels])
  print(rouge_result)
  rouge_list.append(rouge_result)

{'rouge1': 0.8823529411764706, 'rouge2': 0.8118811881188118, 'rougeL': 0.8725490196078433, 'rougeLsum': 0.8867924528301887}
{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
{'rouge1': 0.7526881720430108, 'rouge2': 0.5543478260869565, 'rougeL': 0.5698924731182795, 'rougeLsum': 0.7422680412371134}
{'rouge1': 0.9010989010989011, 'rouge2': 0.7888888888888889, 'rougeL': 0.8461538461538461, 'rougeLsum': 0.8947368421052632}
{'rouge1': 0.8128342245989305, 'rouge2': 0.627027027027027, 'rougeL': 0.7272727272727272, 'rougeLsum': 0.8205128205128204}
{'rouge1': 0.8559322033898306, 'rouge2': 0.7264957264957266, 'rougeL': 0.7966101694915253, 'rougeLsum': 0.8442622950819672}
{'rouge1': 0.8152173913043479, 'rouge2': 0.6263736263736264, 'rougeL': 0.717391304347826, 'rougeLsum': 0.7604166666666666}
{'rouge1': 0.7593582887700534, 'rouge2': 0.508108108108108, 'rougeL': 0.641711229946524, 'rougeLsum': 0.7384615384615386}
{'rouge1': 0.8135593220338982, 'rouge2': 0.5897435897435899, 'rougeL': 0

In [51]:
print(codet5p_tokenizer.batch_decode(preds[0], skip_special_tokens=True))

['', '', 'c', ' r', ' o', ' s', ' s', ' -', ' s', ' i', ' t', ' e', '  ', ' s', ' c', ' r', ' i', ' p', ' t', ' i', ' n', ' g', '  ', ' (', ' x', ' s', ' s', '  ', ' )', '']


In [52]:
print(codet5p_tokenizer.batch_decode(labels[0], skip_special_tokens=True))

['', 'c', ' r', ' o', ' s', ' s', ' -', ' s', ' i', ' t', ' e', '  ', ' s', ' c', ' r', ' i', ' p', ' t', ' i', ' n', ' g', '  ', ' (', ' x', ' s', ' s', '  ', ' )', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [57]:
rouge1_values = [d['rouge1'] for d in rouge_list]
rouge2_values = [d['rouge2'] for d in rouge_list]
rougeL_values = [d['rougeL'] for d in rouge_list]
rouge1_values = sum(rouge1_values) / len(rougeL_values)
average_rouge2 = sum(rouge2_values) / len(rougeL_values)
average_rougeL = sum(rougeL_values) / len(rougeL_values)

rouge1_values, average_rouge2, average_rougeL

(0.8470590127401693, 0.6822108285571038, 0.7416228210213243)